# NKSOPS_03_DataTransformation_04

In [ ]:
#load Enviornment
%run NKSOPS_02_ClassAndMethods_01.ipynb

In [ ]:
# import win32com.client
# import win32com
# import os
# import sys

# f = open("testfile.txt","w+")

# outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
# accounts= win32com.client.Dispatch("Outlook.Application").Session.Accounts;

# def emailleri_al(folder):
#     messages = folder.Items
#     a=len(messages)
#     if a>0:
#         for message2 in messages:
#             try:
#                 sender = message2.SenderEmailAddress
#                 if sender != "":
#                     print(sender, file=f)
#             except:
#                 print("Error")
#                 print(account.DeliveryStore.DisplayName)
#                 pass

#             try:
#                 message2.Save
#                 message2.Close(0)
#             except:
#                  pass



# for account in accounts:
#     global inbox
#     inbox = outlook.Folders(account.DeliveryStore.DisplayName)
#     print("****Account Name**********************************",file=f)
#     print(account.DisplayName,file=f)
#     print(account.DisplayName)
#     print("***************************************************",file=f)
#     folders = inbox.Folders

#     for folder in folders:
#         print("****Folder Name**********************************", file=f)
#         print(folder, file=f)
#         print("*************************************************", file=f)
#         emailleri_al(folder)
#         a = len(folder.folders)

#         if a>0 :
#             global z
#             z = outlook.Folders(account.DeliveryStore.DisplayName).Folders(folder.name)
#             x = z.Folders
#             for y in x:
#                 emailleri_al(y)
#                 print("****Folder Name**********************************", file=f)
#                 print("..."+y.name,file=f)
#                 print("*************************************************", file=f)



# print("Finished Succesfully")

In [ ]:
# from win32com.client import Dispatch
# import win32com.client
# import win32com
# import os
# import sys

# outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
# root_folder = outlook.GetDefaultFolder(2)
# inbox = root_folder.Folders['TEST_OPS']
# messages = inbox.Items
# '''message = messages.GetLast()
# body_content = message.Body
# subject = message.Subject
# categories = message.Categories
# print(body_content)
# print(subject)
# print(categories)
# print msg.SenderName
# print msg.SenderEmailAddress
# print msg.SentOn
# print msg.To
# print msg.CC
# print msg.BCC
# print msg.Subject
# print msg.Body'''
# body_content = message.Subject
# rcvd = message.ReceivedTime
# for message in messages:
#     print(rcvd, message.Subject, body_content)


In [ ]:
# Initalize
NKS_OPS_OUTLOOK_EXPORTS_INFO = NKS_OPS(input_path = r"C:\Users\76350\Documents\NKS_OPS\03_Data\01_Raw_Data\DataFiles\MXEmail\*.csv"
                                   , export_parent_dir = r"C:\Users\76350\Documents\NKS_OPS\03_Data\02_Processed_Data"
                                   , input_columns = None
                                   , export_columns = ['Body',"Unique_ID","From: (Name)","FILE_PATH"]
                                   , research_topic = 'NKS_OPS_Data_Study'
                                   , severity = 'H'
                                   , logfile = False
                                   , sqlite_create = r"C:\Users\76350\Documents\NKS_OPS\03_Data\03_Cleaned_Data\DataBase\NKS_OPS_DB_sqlite.db")

In [ ]:
nks_ops = pd.read_excel(r"C:\Users\76350\Documents\NKS_OPS\03_Data\01_Raw_Data\DataBase\TEST_OPS.xlsx")
nks_ops = nks_ops[['Received','Contents']].rename_axis('ID').reset_index()

In [ ]:
#2 Parse and Extract Sections of Contents and create New columns

#2.a MESSAGE_TYPE
NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=nks_ops, newcolumn='MESSAGETYPE', textcolumn='Contents', start='MX', end='MESSAGE')
nks_ops.loc[:,'MESSAGETYPE'] = 'MX' + nks_ops.loc[:,'MESSAGETYPE'] + 'MESSAGE'
grouped_messages = nks_ops.loc[:,'MESSAGETYPE'].dropna().unique()

new_df = pd.DataFrame()
for message in grouped_messages:
    data_item = nks_ops.copy()
    data_item = data_item[data_item.loc[:,'MESSAGETYPE'].str.contains(message)]
    
    if message == "MX AIRCRAFT OOS MESSAGE":
        #2.b Top Section of Message
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DATE', textcolumn='Contents', start='DATE: ', end='TAIL: ')
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='TAIL', textcolumn='Contents', start='TAIL: ', end='STATION: ').str[:3]
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='STATION', textcolumn='Contents', start='STATION: ', end='DESCRIPTION: ').str[:3]
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DESCRIPTION', textcolumn='Contents', start='DESCRIPTION: ', end=r"STD..GMT...")
        data_item.loc[:,'TROUBLESHOOTING'] = ''
        data_item.loc[:,'CORRECTIVEACTION'] = ''

        #2.c Bottom Section of Message
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='STD_GMT', textcolumn='Contents', start= r"STD..GMT...", end=r"ETRS..LT...")
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='ETRS_DATE', textcolumn='Contents', start=r"ETRS..LT...", end='DESTINATION: ')
        data_item.loc[:,'RTS_DATE'] = ''
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DESTINATION', textcolumn='Contents', start="DESTINATION: ", end="\r\n")
           
        #2.d Make columns visible in csv
        data_item.loc[:,'TAIL'] = data_item.loc[:,'TAIL'].str[:3]
        data_item.loc[:,'STATION'] = data_item.loc[:,'STATION'].str[:3]
        new_df= new_df.append(data_item,ignore_index=True)
        
    if message == "MX AIRCRAFT OOS UPDATE MESSAGE":

        #2.e Top Section of Message
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DATE', textcolumn='Contents', start='DATE: ', end='TAIL: ')
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='TAIL', textcolumn='Contents', start='TAIL: ', end='STATION: ').str[:3]
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='STATION', textcolumn='Contents', start='STATION: ', end='DESCRIPTION: ').str[:3]
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DESCRIPTION', textcolumn='Contents', start='DESCRIPTION: ', end='TROUBLESHOOTING: ')
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='TROUBLESHOOTING', textcolumn='Contents', start='TROUBLESHOOTING: ', end=r"STD..GMT...")
        data_item.loc[:,'CORRECTIVEACTION'] = ''

        #2.f Bottom Section of Message
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='STD_GMT', textcolumn='Contents', start= r"STD..GMT...", end=r"ETRS..LT...")
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='ETRS_DATE', textcolumn='Contents', start=r"ETRS..LT...", end='DESTINATION: ')
        data_item.loc[:,'RTS_DATE'] = ''
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DESTINATION', textcolumn='Contents', start="DESTINATION: ", end="\r\n")
        
        #2.g Make columns visible in csv
        data_item.loc[:,'TAIL'] = data_item.loc[:,'TAIL'].str[:3]
        data_item.loc[:,'STATION'] = data_item.loc[:,'STATION'].str[:3]
        new_df= new_df.append(data_item,ignore_index=True)
              
    if message == "MX AIRCRAFT OOS RETURN TO SERVICE MESSAGE":

        #2.h Top Section of Message
        data_item.loc[:,'DATE'] = ''
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='TAIL', textcolumn='Contents', start='TAIL: ', end='STATION: ').str[:3]
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='STATION', textcolumn='Contents', start='STATION: ', end='DESCRIPTION: ').str[:3]
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DESCRIPTION', textcolumn='Contents', start='DESCRIPTION: ', end='CORRECTIVE ACTION: ')
        data_item.loc[:,'TROUBLESHOOTING'] = ''
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='CORRECTIVEACTION', textcolumn='Contents', start='CORRECTIVE ACTION: ', end=r"STD..GMT...")

        #2.i Bottom Section of Message
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='STD_GMT', textcolumn='Contents', start=r"STD..GMT...", end='RTS DATE: ')
        data_item.loc[:,'ETRS_DATE'] = ''
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='RTS_DATE', textcolumn='Contents', start='RTS DATE: ', end='DESTINATION: ')
        NKS_OPS_OUTLOOK_EXPORTS_INFO.Extract_Between_Text(df=data_item, newcolumn='DESTINATION', textcolumn='Contents', start='DESTINATION: ', end="\r\n")
        
        #2.j Make columns visible in csv
        data_item.loc[:,'TAIL'] = data_item.loc[:,'TAIL'].str[:3]
        data_item.loc[:,'STATION'] = data_item.loc[:,'STATION'].str[:3]
        new_df= new_df.append(data_item,ignore_index=True)




In [ ]:
#2.k Export results to csv
new_df['Unique_ID'] = new_df['ID'].astype(str)+ "_" + new_df['MESSAGETYPE'].astype(str) + "_" + new_df['TAIL'].astype(str) + "_" + new_df['STATION'].astype(str) + "_" + new_df['DESTINATION'].astype(str)+ "_" + new_df['Received'].astype(str).str.replace('-','').str.replace(':','').str.replace(' ','')
new_df['Unique_ID'] = new_df['Unique_ID'].str.replace(' ','')
new_df['DATE'] = new_df['DATE'].str.replace('\n','')
new_df['STD_GMT'] = new_df['STD_GMT'].str.replace('\n','')
new_df['ETRS_DATE'] = new_df['ETRS_DATE'].str.replace('\n','')
new_df['RTS_DATE'] = new_df['RTS_DATE'].str.replace('\n','')
new_df = new_df.sort_values(by=['TAIL','Received'],ascending=True)

new_df.loc[new_df['MESSAGETYPE'].str.contains(r"MX AIRCRAFT OOS RETURN TO SERVICE MESSAGE", na=False), 'DATE'] = new_df['DATE'].shift(1)
new_df[['DATE_DATES','DATE_TIME']] = new_df['DATE'].str.split(' ', expand=True)
new_df['DATE_DATES'] = pd.to_datetime(new_df['DATE_DATES'], format="%d%b%y" ,exact=False,infer_datetime_format=True)
new_df[['DATE_YEAR','DATE_MONTH','DATE_DAY']] = new_df['DATE_DATES'].astype(str).str.split('-', expand=True)
new_df['DATE_YEAR'] = new_df['DATE_YEAR'].replace({'2019':'19', '2018':'18'})


new_df['Unique_Flight_Deprt'] = new_df.DATE_MONTH +"_"+ new_df.DATE_DAY +"_"+ new_df.DATE_YEAR +"_"+ new_df.STATION +"_"+ new_df.TAIL
new_df['Unique_Flight_Arrvl'] = new_df.DATE_MONTH +"_"+ new_df.DATE_DAY +"_"+ new_df.DATE_YEAR +"_"+ new_df.STATION +"_"+ new_df.TAIL
move_col = new_df.columns.tolist()
move_col.insert(1, move_col.pop(move_col.index('Unique_ID')))
new_df = new_df.reindex(columns= move_col)

new_df['Event_Order'] =''
for index, row in new_df.iterrows():
    new_df.loc[new_df['MESSAGETYPE'].str.contains(r"MX AIRCRAFT OOS MESSAGE", na=False), 'Event_Order'] = "DESCRIPTION: " + new_df['DESCRIPTION'].astype(str)
    new_df.loc[new_df['MESSAGETYPE'].str.contains(r"MX AIRCRAFT OOS UPDATE MESSAGE", na=False), 'Event_Order'] =  new_df['Event_Order'].shift(1).astype(str) + "DESCRIPTION: " + new_df['DESCRIPTION'].astype(str) + "TROUBLESHOOTING: " + new_df['TROUBLESHOOTING'].astype(str) 
    new_df.loc[new_df['MESSAGETYPE'].str.contains(r"MX AIRCRAFT OOS RETURN TO SERVICE MESSAGE", na=False), 'Event_Order'] = new_df['Event_Order'].shift(1).astype(str) + "CORRECTIVEACTION: " + new_df['CORRECTIVEACTION']

new_df = new_df.drop(columns = ['ID','DATE_DATES','DATE_TIME','DATE_YEAR','DATE_MONTH','DATE_DAY'])
new_df.to_csv(r'C:\Users\76350\Documents\NKS_OPS\03_Data\03_Cleaned_Data\DataFiles\Finial.csv', index=False)
new_df

In [ ]:
#export to sqlite3
NKS_OPS_OUTLOOK_EXPORTS_INFO.NKS_OPS_DB(db_file=NKS_OPS_OUTLOOK_EXPORTS_INFO.sqlite_create)
num_rows = len(new_df)
conn = NKS_OPS_OUTLOOK_EXPORTS_INFO.create_connection(db_file = NKS_OPS_OUTLOOK_EXPORTS_INFO.sqlite_create)
cur = conn.cursor()
for opsrow in range(num_rows):
    try:
        new_df.iloc[opsrow:opsrow+1].to_sql(name='NKS_OPS', con=conn, if_exists='append', chunksize=10000, index=False)
    except exc.IntegrityError:
        #Ignore duplicates
        pass

In [ ]:
conn.close()